# Introduction

This notebook demonstrates two simple simulation and analysis workflows. First, `hoomd_flowws` is used to perform a simulation of rounded cubes with attractive interactions using DEM-MD. We then visualize the system using `flowws-analysis` and `flowws-freud`, coloring particles by their potential energy.

## Plotting potential

We'll use an isotropic Gaussian potential from the centers of particles to define the attraction. First we plot the potential to get an idea of how the Gaussian parameters translate into interaction energies at relevant distances given the geometry of our DEM particles.

The cubes we use will have a side length of 1.5, then be rounded by a sphere of radius 0.5.

In [ ]:
import ipywidgets
import matplotlib, matplotlib.pyplot as pp
import numpy as np

shape = dict(type='cube',
             modifications=[
                 dict(type='scale', factor=1.5),
                 dict(type='round', radius=.5),
             ])
epsilon = -20.
sigma = 2.
rcut = 8.
@ipywidgets.interact
def plot(epsilon=epsilon, sigma=sigma):
    # particle center-center distance when DEM cubes are face-to-face
    cube_nearest = shape['modifications'][0]['factor'] + 2*shape['modifications'][1]['radius']
    # particle center-center distance when DEM cubes are corner-to-corner
    cube_farthest = np.sqrt(3)*shape['modifications'][0]['factor'] + 2*shape['modifications'][1]['radius']

    rs = np.linspace(cube_nearest, rcut, 512)
    Us = epsilon*np.exp(-.5*(rs/sigma)**2)
    pp.plot(rs, Us)
    pp.vlines(cube_farthest, np.min(Us), 0)
    pp.xlabel('r'); pp.ylabel('U')

## Simulation

We let particles thermalize without the attractive interactions before turning on the Gaussian potentials. To color particles by their potential energy during visualization, we also need to save the potential for each particle at each frame we dump.

In [ ]:
import flowws

# this line can be used to clear the trajectory
# and backup files that have been generated
# !rm -f 3d_sticky_cubes.*

scope = dict(
    dump_filename='3d_sticky_cubes.sqlite',
)

from hoomd_flowws import *

workflow = flowws.Workflow([
    ShapeDefinition(shape_arguments=[shape]),
    Init(number=256),
    DEMInteraction(type='wca'),
    Run(steps=1e4, integrator='langevin'),
    Interaction(type='gauss', global_params=dict(r_cut=rcut).items(),
                pair_params=[('A', 'A', 'epsilon', epsilon), ('A', 'A', 'sigma', sigma)]),
    Run(steps=4e4, integrator='langevin', dump_period=4e3, expand_by=1.5, ramp_t_to=1e-2,
        trajectory_quantities=['potential_energy']),
], flowws.DirectoryStorage(), scope)

workflow.run()

# Visualization

We show three different ways to visualize the system. Select one depending on which mode best suits your use case.

## Directly within notebook

This mode is convenient for quick analysis, but depends on having vispy's notebook integration components [properly installed](http://vispy.org/installation.html).

In [ ]:
from flowws_analysis import *
from flowws_freud import *

flowws.Workflow([
    GTAR(filename=scope['dump_filename']),
    RDF(r_max=9),
    Colormap(argument='potential_energy'),
    SmoothBOD(r_max=4),
    Center(),
    Plato(),
    ViewNotebook(),
], flowws.DirectoryStorage(), scope).run()

## Controls in notebook, plato visuals in popup windows

This mode is less reliant on details of your vispy installation. Jupyter is used to launch a local qt5 event loop, then visuals are made to pop up in their own windows while controls stay in the jupyter notebook. This depends on having the jupyter server running on your local machine in order to control windows.

In [ ]:
from flowws_analysis import *
from flowws_freud import *

%gui qt5

flowws.Workflow([
    GTAR(filename=scope['dump_filename']),
    RDF(r_max=9),
    Colormap(argument='potential_energy'),
    SmoothBOD(r_max=4),
    Center(),
    Plato(),
    ViewNotebook(vispy_backend='pyside2'),
], flowws.DirectoryStorage(), scope).run()

## Standalone GUI window

This mode launches its own window showing all controls and analysis using the `ViewQt` module available in `flowws-analysis`. Of course, this command can also be run in the command line without a notebook.

In [ ]:
%%script sh

python -m flowws.run \
    GTAR --filename 3d_sticky_cubes.sqlite \
    RDF --r-max 9 \
    Colormap --argument potential_energy \
    SmoothBOD --r-max 4 \
    Center \
    Plato \
    ViewQt